## Car Price Assignment (Model building using 'Multiple Linear Regression')

#### Problem Statement:

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts. 

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

- Which variables are significant in predicting the price of a car
- How well those variables describe the price of a car

Based on various market surveys, the consulting firm has gathered a large dataset of different types of cars across the Americal market. 

## Step 1: Reading and Understanding the Data

Let us first import NumPy and Pandas and read the CarPrice dataset

In [1]:
import numpy as np
import pandas as pd

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Read csv dataset
car_price = pd.read_csv("CarPrice_Assignment.csv")

# Check the head of the dataset
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


#### Data Preparation

- There is a variable named CarName which is comprised of two parts - the first word is the name of 'car company' and the second is the 'car model'. For example, chevrolet impala has 'chevrolet' as the car company name and 'impala' as the car model name. You need to consider only company name as the independent variable for model building.

In [2]:
car_price['CarName'] = car_price['CarName'].str.replace('-',' ')
car_price['CarCompany'] = car_price['CarName'].str.split(" ", n = 1, expand = True)[0]
# Dropping CarName and car_ID columns
car_price.drop(columns =["CarName", "car_ID"], inplace = True) 
# Check the head of the dataset
car_price.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,CarCompany
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


#### Inspect the various aspects of the car_price dataframe to see if there is missing data

In [3]:
car_price.shape

(205, 25)

In [4]:
car_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 25 columns):
symboling           205 non-null int64
fueltype            205 non-null object
aspiration          205 non-null object
doornumber          205 non-null object
carbody             205 non-null object
drivewheel          205 non-null object
enginelocation      205 non-null object
wheelbase           205 non-null float64
carlength           205 non-null float64
carwidth            205 non-null float64
carheight           205 non-null float64
curbweight          205 non-null int64
enginetype          205 non-null object
cylindernumber      205 non-null object
enginesize          205 non-null int64
fuelsystem          205 non-null object
boreratio           205 non-null float64
stroke              205 non-null float64
compressionratio    205 non-null float64
horsepower          205 non-null int64
peakrpm             205 non-null int64
citympg             205 non-null int64
highwaympg    

In [5]:
car_price.describe()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


## Step 2: Visualising the Data

Let's now spend some time doing what is arguably the most important step - **understanding the data**.
- If there is some obvious multicollinearity going on, this is the first place to catch it
- Here's where you'll also identify if some predictors directly have a strong association with the outcome variable

We'll visualise our data using `matplotlib` and `seaborn`.

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

#### Visualising Numeric Variables

Let's make a pairplot of all the numeric variables

In [7]:
sns.pairplot(car_price)
plt.show()

#### Visualising Categorical Variables

As there are a few categorical variables as well. Let's make a boxplot for some of these variables.

In [8]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)
sns.boxplot(x = 'fueltype', y = 'price', data = car_price)
plt.subplot(2,3,2)
sns.boxplot(x = 'doornumber', y = 'price', data = car_price)
plt.subplot(2,3,3)
sns.boxplot(x = 'carbody', y = 'price', data = car_price)
plt.subplot(2,3,4)
sns.boxplot(x = 'drivewheel', y = 'price', data = car_price)
plt.subplot(2,3,5)
sns.boxplot(x = 'enginelocation', y = 'price', data = car_price)
plt.subplot(2,3,6)
sns.boxplot(x = 'cylindernumber', y = 'price', data = car_price)
plt.show()

## Step 3: Data Preparation
#### Creating dummy variables for each categorical columns

- fueltype, aspiration, doornumber, carbody, drivewheel, enginelocation, enginetype, cylindernumber, fuelsystem, CarCompany

In [9]:
# Let's create dummy variable(s) for 'fueltype' column and drop the first column from dataframe using 'drop_first = True'
fueltype = pd.get_dummies(car_price['fueltype'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, fueltype], axis = 1)
# Drop 'fueltype' column as we have created the dummies for it
car_price.drop(['fueltype'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,CarCompany,gas
0,3,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,...,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa,1
1,3,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,...,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa,1
2,1,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,...,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa,1
3,2,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,...,3.19,3.40,10.0,102,5500,24,30,13950.0,audi,1
4,2,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,...,3.19,3.40,8.0,115,5500,18,22,17450.0,audi,1


In [10]:
# Let's create dummy variable(s) for 'aspiration' column and drop the first column from dataframe using 'drop_first = True'
aspiration = pd.get_dummies(car_price['aspiration'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, aspiration], axis = 1)
# Drop 'aspiration' column as we have created the dummies for it
car_price.drop(['aspiration'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,...,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,CarCompany,gas,turbo
0,3,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,...,2.68,9.0,111,5000,21,27,13495.0,alfa,1,0
1,3,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,...,2.68,9.0,111,5000,21,27,16500.0,alfa,1,0
2,1,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,...,3.47,9.0,154,5000,19,26,16500.0,alfa,1,0
3,2,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,...,3.40,10.0,102,5500,24,30,13950.0,audi,1,0
4,2,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,...,3.40,8.0,115,5500,18,22,17450.0,audi,1,0


In [11]:
# Let's create dummy variable(s) for 'doornumber' column and drop the first column from dataframe using 'drop_first = True'
doornumber = pd.get_dummies(car_price['doornumber'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, doornumber], axis = 1)
# Drop 'doornumber' column as we have created the dummies for it
car_price.drop(['doornumber'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,...,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,CarCompany,gas,turbo,two
0,3,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,...,9.0,111,5000,21,27,13495.0,alfa,1,0,1
1,3,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,...,9.0,111,5000,21,27,16500.0,alfa,1,0,1
2,1,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,...,9.0,154,5000,19,26,16500.0,alfa,1,0,1
3,2,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,...,10.0,102,5500,24,30,13950.0,audi,1,0,0
4,2,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,...,8.0,115,5500,18,22,17450.0,audi,1,0,0


In [12]:
# Let's create dummy variable(s) for 'carbody' column and drop the first column from dataframe using 'drop_first = True'
carbody = pd.get_dummies(car_price['carbody'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, carbody], axis = 1)
# Drop 'carbody' column as we have created the dummies for it
car_price.drop(['carbody'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,...,highwaympg,price,CarCompany,gas,turbo,two,hardtop,hatchback,sedan,wagon
0,3,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,...,27,13495.0,alfa,1,0,1,0,0,0,0
1,3,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,...,27,16500.0,alfa,1,0,1,0,0,0,0
2,1,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,...,26,16500.0,alfa,1,0,1,0,1,0,0
3,2,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,...,30,13950.0,audi,1,0,0,0,0,1,0
4,2,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,...,22,17450.0,audi,1,0,0,0,0,1,0


In [13]:
# Let's create dummy variable(s) for 'drivewheel' column and drop the first column from dataframe using 'drop_first = True'
drivewheel = pd.get_dummies(car_price['drivewheel'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, drivewheel], axis = 1)
# Drop 'drivewheel' column as we have created the dummies for it
car_price.drop(['drivewheel'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,...,CarCompany,gas,turbo,two,hardtop,hatchback,sedan,wagon,fwd,rwd
0,3,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,...,alfa,1,0,1,0,0,0,0,0,1
1,3,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,...,alfa,1,0,1,0,0,0,0,0,1
2,1,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,...,alfa,1,0,1,0,1,0,0,0,1
3,2,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,...,audi,1,0,0,0,0,1,0,1,0
4,2,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,...,audi,1,0,0,0,0,1,0,0,0


In [14]:
# Let's create dummy variable(s) for 'enginelocation' column and drop the first column from dataframe using 'drop_first = True'
enginelocation = pd.get_dummies(car_price['enginelocation'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, enginelocation], axis = 1)
# Drop 'enginelocation' column as we have created the dummies for it
car_price.drop(['enginelocation'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,...,gas,turbo,two,hardtop,hatchback,sedan,wagon,fwd,rwd,rear
0,3,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,...,1,0,1,0,0,0,0,0,1,0
1,3,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,...,1,0,1,0,0,0,0,0,1,0
2,1,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,...,1,0,1,0,1,0,0,0,1,0
3,2,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,...,1,0,0,0,0,1,0,1,0,0
4,2,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,...,1,0,0,0,0,1,0,0,0,0


In [15]:
# Let's create dummy variable(s) for 'enginetype' column and drop the first column from dataframe using 'drop_first = True'
enginetype = pd.get_dummies(car_price['enginetype'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, enginetype], axis = 1)
# Drop 'enginetype' column as we have created the dummies for it
car_price.drop(['enginetype'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,fuelsystem,boreratio,...,wagon,fwd,rwd,rear,dohcv,l,ohc,ohcf,ohcv,rotor
0,3,88.6,168.8,64.1,48.8,2548,four,130,mpfi,3.47,...,0,0,1,0,0,0,0,0,0,0
1,3,88.6,168.8,64.1,48.8,2548,four,130,mpfi,3.47,...,0,0,1,0,0,0,0,0,0,0
2,1,94.5,171.2,65.5,52.4,2823,six,152,mpfi,2.68,...,0,0,1,0,0,0,0,0,1,0
3,2,99.8,176.6,66.2,54.3,2337,four,109,mpfi,3.19,...,0,1,0,0,0,0,1,0,0,0
4,2,99.4,176.6,66.4,54.3,2824,five,136,mpfi,3.19,...,0,0,0,0,0,0,1,0,0,0


In [16]:
# Let's create dummy variable(s) for 'cylindernumber' column and drop the first column from dataframe using 'drop_first = True'
cylindernumber = pd.get_dummies(car_price['cylindernumber'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, cylindernumber], axis = 1)
# Drop 'cylindernumber' column as we have created the dummies for it
car_price.drop(['cylindernumber'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,fuelsystem,boreratio,stroke,...,ohc,ohcf,ohcv,rotor,five,four,six,three,twelve,two
0,3,88.6,168.8,64.1,48.8,2548,130,mpfi,3.47,2.68,...,0,0,0,0,0,1,0,0,0,0
1,3,88.6,168.8,64.1,48.8,2548,130,mpfi,3.47,2.68,...,0,0,0,0,0,1,0,0,0,0
2,1,94.5,171.2,65.5,52.4,2823,152,mpfi,2.68,3.47,...,0,0,1,0,0,0,1,0,0,0
3,2,99.8,176.6,66.2,54.3,2337,109,mpfi,3.19,3.40,...,1,0,0,0,0,1,0,0,0,0
4,2,99.4,176.6,66.4,54.3,2824,136,mpfi,3.19,3.40,...,1,0,0,0,1,0,0,0,0,0


In [17]:
# Let's create dummy variable(s) for 'fuelsystem' column and drop the first column from dataframe using 'drop_first = True'
fuelsystem = pd.get_dummies(car_price['fuelsystem'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, fuelsystem], axis = 1)
# Drop 'fuelsystem' column as we have created the dummies for it
car_price.drop(['fuelsystem'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,three,twelve,two,2bbl,4bbl,idi,mfi,mpfi,spdi,spfi
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,0,0,0,0,0,0,0,1,0,0
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,1,0,0
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,1,0,0


In [18]:
# Let's create dummy variable(s) for 'CarCompany' column and drop the first column from dataframe using 'drop_first = True'
CarCompany = pd.get_dummies(car_price['CarCompany'], drop_first = True)
# Add the results to the original car_price dataframe
car_price = pd.concat([car_price, CarCompany], axis = 1)
# Drop 'CarCompany' column as we have created the dummies for it
car_price.drop(['CarCompany'], axis = 1, inplace = True)
# Now let's see the head of our dataframe.
car_price.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,porsche,renault,saab,subaru,toyota,toyouta,vokswagen,volkswagen,volvo,vw
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,0,0,0
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,0,0,0
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,0,0,0,0,0,0,0,0,0,0
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,0,0,0
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,0,0,0


#### Check now if there are any categorical columns before splitting data

In [19]:
car_price.shape

(205, 71)

In [20]:
car_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 71 columns):
symboling           205 non-null int64
wheelbase           205 non-null float64
carlength           205 non-null float64
carwidth            205 non-null float64
carheight           205 non-null float64
curbweight          205 non-null int64
enginesize          205 non-null int64
boreratio           205 non-null float64
stroke              205 non-null float64
compressionratio    205 non-null float64
horsepower          205 non-null int64
peakrpm             205 non-null int64
citympg             205 non-null int64
highwaympg          205 non-null int64
price               205 non-null float64
gas                 205 non-null uint8
turbo               205 non-null uint8
two                 205 non-null uint8
hardtop             205 non-null uint8
hatchback           205 non-null uint8
sedan               205 non-null uint8
wagon               205 non-null uint8
fwd                 2

## Step 4: Splitting the Data into Training and Testing Sets

The first basic step for regression is performing a train-test split.

In [21]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
df_train, df_test = train_test_split(car_price, train_size = 0.7, test_size = 0.3, random_state = 100)

### Rescaling the Features 

Scaling doesn't impact the model. It is extremely important to rescale the variables so that they have a comparable scale. If we don't have comparable scales, then some of the coefficients as obtained by fitting the regression model might be very large or very small as compared to the other coefficients. This might become very annoying at the time of model evaluation. So it is important to use standardization or normalization so that the units of the coefficients obtained are all on the same scale. There are two common ways of rescaling:

1. Min-Max scaling 
2. Standardisation (mean-0, sigma-1) 

Here, i am using MinMax scaling.

In [22]:
from sklearn.preprocessing import MinMaxScaler

In [23]:
scaler = MinMaxScaler()

In [24]:
# Apply scaler() to all the columns except the 'boolean(yes/no)' and 'dummy' variables
num_vars = ['symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 
            'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_train.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,porsche,renault,saab,subaru,toyota,toyouta,vokswagen,volkswagen,volvo,vw
122,0.6,0.244828,0.426016,0.291667,0.265487,0.272692,0.139623,0.230159,0.525253,0.15000,...,0,0,0,0,0,0,0,0,0,0
125,1.0,0.272414,0.452033,0.666667,0.212389,0.500388,0.339623,1.000000,0.464646,0.15625,...,1,0,0,0,0,0,0,0,0,0
166,0.6,0.272414,0.448780,0.308333,0.424779,0.314973,0.139623,0.444444,0.449495,0.15000,...,0,0,0,0,1,0,0,0,0,0
1,1.0,0.068966,0.450407,0.316667,0.088496,0.411171,0.260377,0.626984,0.247475,0.12500,...,0,0,0,0,0,0,0,0,0,0
199,0.2,0.610345,0.775610,0.575000,0.858407,0.647401,0.260377,0.746032,0.484848,0.03125,...,0,0,0,0,0,0,0,0,1,0


In [25]:
df_train.describe()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,porsche,renault,saab,subaru,toyota,toyouta,vokswagen,volkswagen,volvo,vw
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,...,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.0,143.000000,143.000000,143.000000
mean,0.559441,0.411141,0.525476,0.461655,0.509004,0.407878,0.241351,0.497946,0.535389,0.190787,...,0.020979,0.013986,0.020979,0.076923,0.132867,0.006993,0.0,0.034965,0.041958,0.013986
std,0.239200,0.205581,0.204848,0.184517,0.215378,0.211269,0.154619,0.207140,0.157843,0.237181,...,0.143818,0.117845,0.143818,0.267406,0.340624,0.083624,0.0,0.184337,0.201198,0.117845
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.400000,0.272414,0.399187,0.304167,0.353982,0.245539,0.135849,0.305556,0.464646,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,0.600000,0.341379,0.502439,0.425000,0.522124,0.355702,0.184906,0.500000,0.545455,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,0.600000,0.503448,0.669919,0.550000,0.668142,0.559542,0.301887,0.682540,0.611111,0.150000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


In [26]:
# Let's check the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (16, 10))
sns.heatmap(df_train.corr())
plt.show()

### Dividing data into X and Y sets for the model building

In [27]:
y_train = df_train.pop('price')
X_train = df_train

## Step 5: Building a linear model
We will be using the **LinearRegression function from SciKit Learn** for its compatibility with **RFE** (**Recursive Feature Elimination**, which is a utility from sklearn)

In [28]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [29]:
# Running RFE with the output number of the variable equal to 20
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 20)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [30]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

[('symboling', False, 34),
 ('wheelbase', False, 7),
 ('carlength', False, 11),
 ('carwidth', True, 1),
 ('carheight', False, 8),
 ('curbweight', True, 1),
 ('enginesize', True, 1),
 ('boreratio', True, 1),
 ('stroke', True, 1),
 ('compressionratio', False, 15),
 ('horsepower', False, 5),
 ('peakrpm', False, 6),
 ('citympg', False, 29),
 ('highwaympg', False, 16),
 ('gas', False, 14),
 ('turbo', False, 4),
 ('two', False, 42),
 ('hardtop', False, 21),
 ('hatchback', False, 20),
 ('sedan', False, 22),
 ('wagon', False, 23),
 ('fwd', False, 44),
 ('rwd', False, 32),
 ('rear', True, 1),
 ('dohcv', False, 2),
 ('l', True, 1),
 ('ohc', False, 43),
 ('ohcf', True, 1),
 ('ohcv', False, 31),
 ('rotor', True, 1),
 ('five', True, 1),
 ('four', True, 1),
 ('six', False, 18),
 ('three', True, 1),
 ('twelve', True, 1),
 ('two', True, 1),
 ('2bbl', False, 39),
 ('4bbl', False, 24),
 ('idi', False, 19),
 ('mfi', False, 46),
 ('mpfi', False, 26),
 ('spdi', False, 25),
 ('spfi', False, 45),
 ('alfa', F

In [31]:
col = X_train.columns[rfe.support_]
col

Index(['carwidth', 'curbweight', 'enginesize', 'boreratio', 'stroke', 'rear',
       'l', 'ohcf', 'rotor', 'five', 'four', 'three', 'twelve', 'two', 'bmw',
       'peugeot', 'porsche', 'saab', 'subaru', 'volvo'],
      dtype='object')

In [32]:
X_train.columns[~rfe.support_]

Index(['symboling', 'wheelbase', 'carlength', 'carheight', 'compressionratio',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'gas', 'turbo', 'two',
       'hardtop', 'hatchback', 'sedan', 'wagon', 'fwd', 'rwd', 'dohcv', 'ohc',
       'ohcv', 'six', '2bbl', '4bbl', 'idi', 'mfi', 'mpfi', 'spdi', 'spfi',
       'alfa', 'audi', 'buick', 'chevrolet', 'dodge', 'honda', 'isuzu',
       'jaguar', 'maxda', 'mazda', 'mercury', 'mitsubishi', 'nissan',
       'plymouth', 'porcshce', 'renault', 'toyota', 'toyouta', 'vokswagen',
       'volkswagen', 'vw'],
      dtype='object')

### Building models using statsmodel, for the detailed statistics
#### Building first model

In [33]:
# Creating X_train dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [34]:
# Adding a constant variable
import statsmodels.api as sm
X_train_rfe = sm.add_constant(X_train_rfe)
# Running the linear model
lm = sm.OLS(y_train,X_train_rfe).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                     122.9
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           5.46e-70
Time:                        10:32:47   Log-Likelihood:                 226.86
No. Observations:                 143   AIC:                            -415.7
Df Residuals:                     124   BIC:                            -359.4
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3084      0.041     -7.580      0.0

#### Dropping variable 'two' as it has high p-value and updating the model
#### Building second model

In [35]:
# Removing a variable
X_train_new = X_train_rfe.drop(["two"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                     130.5
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           6.19e-71
Time:                        10:32:47   Log-Likelihood:                 226.54
No. Observations:                 143   AIC:                            -417.1
Df Residuals:                     125   BIC:                            -363.8
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3017      0.040     -7.616      0.0

#### Dropping variable 'subaru' as it has high p-value and updating the model
#### Building third model

In [36]:
# Removing a variable
X_train_new = X_train_new.drop(["subaru"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                     130.5
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           6.19e-71
Time:                        10:32:47   Log-Likelihood:                 226.54
No. Observations:                 143   AIC:                            -417.1
Df Residuals:                     125   BIC:                            -363.8
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3017      0.040     -7.616      0.0

#### Dropping variable 'ohcf' as it has high p-value and updating the model
#### Building fourth model

In [37]:
# Removing a variable
X_train_new = X_train_new.drop(["ohcf"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     135.8
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           2.89e-71
Time:                        10:32:48   Log-Likelihood:                 224.57
No. Observations:                 143   AIC:                            -415.1
Df Residuals:                     126   BIC:                            -364.8
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2706      0.036     -7.455      0.0

### Checking VIF

Variance Inflation Factor or VIF, gives a basic quantitative idea about how much the feature variables are correlated with each other. It is an extremely important parameter to test our linear model. The formula for calculating `VIF` is:

### $ VIF_i = \frac{1}{1 - {R_i}^2} $

In [38]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
14,peugeot,inf
11,three,inf
7,l,inf
0,const,65.57
3,enginesize,31.79
10,four,13.73
2,curbweight,12.49
1,carwidth,6.43
4,boreratio,6.02
8,rotor,5.98


#### We generally want a VIF that is less than 5. So there are clearly some variables we need to drop.
#### Dropping variable 'peugeot' which has high VIF values and updating model
#### Building fifth model

In [39]:
# Removing a variable
X_train_new = X_train_new.drop(["peugeot"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     135.8
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           2.89e-71
Time:                        10:32:48   Log-Likelihood:                 224.57
No. Observations:                 143   AIC:                            -415.1
Df Residuals:                     126   BIC:                            -364.8
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2706      0.036     -7.455      0.0

In [40]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
0,const,65.57
3,enginesize,31.79
10,four,13.73
2,curbweight,12.49
1,carwidth,6.43
4,boreratio,6.02
8,rotor,5.98
9,five,3.32
12,twelve,3.17
5,stroke,2.48


#### Dropping variable 'enginesize' which has high VIF values and updating model
#### Building sixth model

In [41]:
# Removing a variable
X_train_new = X_train_new.drop(["enginesize"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.900
Method:                 Least Squares   F-statistic:                     86.57
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           4.59e-59
Time:                        10:32:48   Log-Likelihood:                 189.84
No. Observations:                 143   AIC:                            -347.7
Df Residuals:                     127   BIC:                            -300.3
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1607      0.043     -3.709      0.0

#### Dropping variable 'volvo' as it has high p-value and updating the model
#### Building seventh model

In [42]:
# Removing a variable
X_train_new = X_train_new.drop(["volvo"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     93.47
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           4.68e-60
Time:                        10:32:48   Log-Likelihood:                 189.84
No. Observations:                 143   AIC:                            -349.7
Df Residuals:                     128   BIC:                            -305.2
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1614      0.043     -3.786      0.0

#### Dropping variable 'five' as it has high p-value and updating the model
#### Building eigth model

In [43]:
# Removing a variable
X_train_new = X_train_new.drop(["five"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     101.3
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           4.83e-61
Time:                        10:32:48   Log-Likelihood:                 189.77
No. Observations:                 143   AIC:                            -351.5
Df Residuals:                     129   BIC:                            -310.1
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1640      0.042     -3.918      0.0

#### Dropping variable 'four' as it has high p-value and updating the model
#### Building ninth model

In [44]:
# Removing a variable
X_train_new = X_train_new.drop(["four"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                     110.6
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           4.52e-62
Time:                        10:32:48   Log-Likelihood:                 189.76
No. Observations:                 143   AIC:                            -353.5
Df Residuals:                     130   BIC:                            -315.0
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1679      0.029     -5.876      0.0

#### Dropping variable 'saab' as it has high p-value and updating the model
#### Building tenth model

In [45]:
# Removing a variable
X_train_new = X_train_new.drop(["saab"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                     121.4
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           4.37e-63
Time:                        10:32:48   Log-Likelihood:                 189.66
No. Observations:                 143   AIC:                            -355.3
Df Residuals:                     131   BIC:                            -319.8
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1678      0.028     -5.892      0.0

#### Dropping variable 'stroke' as it has high p-value and updating the model
#### Building eleventh model

In [46]:
# Removing a variable
X_train_new = X_train_new.drop(["stroke"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.910
Model:                            OLS   Adj. R-squared:                  0.904
Method:                 Least Squares   F-statistic:                     134.0
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           4.72e-64
Time:                        10:32:48   Log-Likelihood:                 189.39
No. Observations:                 143   AIC:                            -356.8
Df Residuals:                     132   BIC:                            -324.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1828      0.019     -9.527      0.0

#### Dropping variable 'rotor' as it has high p-value and updating the model
#### Building twelveth model

In [47]:
# Removing a variable
X_train_new = X_train_new.drop(["rotor"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                     147.4
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           1.09e-64
Time:                        10:32:48   Log-Likelihood:                 188.23
No. Observations:                 143   AIC:                            -356.5
Df Residuals:                     133   BIC:                            -326.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1826      0.019     -9.475      0.0

In [48]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
0,const,11.73
2,curbweight,5.38
1,carwidth,4.89
3,boreratio,1.88
9,porsche,1.80
4,rear,1.62
5,l,1.30
6,three,1.25
7,twelve,1.07
8,bmw,1.07


#### Calculating VIF for updated model variables

In [49]:
# Drop contant
X_train_new = X_train_new.drop(["const"], axis = 1)

# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
0,carwidth,25.25
1,curbweight,23.05
2,boreratio,9.73
8,porsche,1.64
3,rear,1.57
4,l,1.32
5,three,1.15
7,bmw,1.12
6,twelve,1.07


#### Dropping variable 'carwidth' which has high VIF values and updating model
#### Building thirteenth model

In [50]:
# Removing a variable
X_train_new = X_train_new.drop(["carwidth"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     133.6
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           5.28e-60
Time:                        10:32:48   Log-Likelihood:                 173.88
No. Observations:                 143   AIC:                            -329.8
Df Residuals:                     134   BIC:                            -303.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1258      0.018     -7.053      0.0

#### Dropping variable 'boreratio' as it has high p-value and updating the model
#### Building fourteenth model

In [51]:
# Removing a variable
X_train_new = X_train_new.drop(["boreratio"], axis = 1)
# Adding a constant variable
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_new)
# Running the linear model
lm = sm.OLS(y_train,X_train_lm).fit()
#Let's see the summary of our linear model
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.880
Method:                 Least Squares   F-statistic:                     149.9
Date:                Thu, 16 Jan 2020   Prob (F-statistic):           1.97e-60
Time:                        10:32:48   Log-Likelihood:                 172.21
No. Observations:                 143   AIC:                            -328.4
Df Residuals:                     135   BIC:                            -304.7
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1449      0.014    -10.087      0.0

In [52]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
6,porsche,1.55
1,rear,1.50
0,curbweight,1.33
2,l,1.31
3,three,1.15
5,bmw,1.09
4,twelve,1.04


## Step 7:Residual Analysis of the train data

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [53]:
y_train_price = lm.predict(X_train_lm)

In [54]:
# Importing the required libraries for plots.
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [55]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_price), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

Text(0.5, 0, 'Errors')

## Step 8: Making Predictions Using the Final Model

Now that we have fitted the model & checked the normality of error terms, its time to go ahead & make predictions using the final, i.e, fourteenth model.

#### Applying the scaling on the test sets

In [56]:
# Apply scaler() to all the columns except the 'boolean(yes/no)' and 'dummy' variables
num_vars = ['symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 
            'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']

df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.describe()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,porsche,renault,saab,subaru,toyota,toyouta,vokswagen,volkswagen,volvo,vw
count,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,...,62.000000,62.0,62.000000,62.000000,62.000000,62.0,62.000000,62.000000,62.000000,62.0
mean,0.583871,0.437764,0.559481,0.480376,0.559663,0.428470,0.265673,0.556580,0.544314,0.209375,...,0.016129,0.0,0.048387,0.016129,0.193548,0.0,0.016129,0.064516,0.080645,0.0
std,0.271724,0.212861,0.189947,0.165440,0.215811,0.179488,0.162820,0.228484,0.160738,0.273690,...,0.127000,0.0,0.216335,0.127000,0.398304,0.0,0.127000,0.247676,0.274512,0.0
min,0.000000,0.000000,0.056911,0.183333,0.141593,0.128394,0.067925,-0.111111,-0.060606,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
25%,0.400000,0.313793,0.459350,0.358333,0.384956,0.305275,0.177358,0.404762,0.484848,0.093750,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
50%,0.600000,0.387931,0.547967,0.441667,0.588496,0.406129,0.226415,0.563492,0.585859,0.125000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
75%,0.800000,0.570690,0.719919,0.516667,0.699115,0.564876,0.316038,0.746032,0.641414,0.150000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
max,1.000000,1.182759,1.089431,0.975000,1.061947,0.935609,0.932075,0.888889,0.863636,1.000000,...,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0


#### Dividing into X_test and y_test

In [57]:
y_test = df_test.pop('price')
X_test = df_test

## Step 9: Model Evaluation

Now let's use our model to make predictions.

In [58]:
# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train_new.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test_new)

# Making predictions
y_pred = lm.predict(X_test_new)

# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label

Text(0, 0.5, 'y_pred')

#### Calculate the R-squared score on the test set.

In [59]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.8070779073959733